# Pricing Action Engine

## Input
- Reads `pricing_with_discount.xlsx` from data_extraction.ipynb

## Output
- `new_price` - Recommended target price
- `sku_discount_flag` - ADD / REMOVE / KEEP / NO
- `qd_discount_flag` - REMOVE_T3 / REMOVE_T2 / REMOVE_T1 / KEEP / NO
- `tier_with_problem` - Which QD tier to remove (T3, T2, T1, or None)
- `new_cart_rule` - New cart rule value
- `action_reason` - Why this action was taken

## Logic Conditions

| # | Condition | Price | SKU Discount | QD | Cart Rule |
|---|-----------|-------|--------------|-----|-----------|
| 1 | Zero Demand + Stock > 0 | -2 steps | ADD | NO | +25% |
| 2 | Star/Over Achiever + Stock > 0 (not zero demand) | +1 step (if both <50%) | REMOVE if >50% | REMOVE highest tier if >50% | -25% |
| 3 | On Track + Stock > 0 (not zero demand) | +1 step | KEEP | KEEP | KEEP |
| 4 | Struggling/Underperforming + Stock > 0 (not zero demand) | -1 step* | ADD | KEEP | +25% |
| 4b | **Critical** + Stock > 0 (not zero demand) | **-2 steps*** | ADD | KEEP | +25% |

*\*Price Oscillation Prevention:*
- *Condition 2/3: Skip price INCREASE if yesterday was Struggling/Underperforming/Critical AND was NOT OOS*
- *Condition 4: Skip price REDUCTION if yesterday was On Track/Over Achiever/Star Performer*
| 5 | No Data + Stock > 0 (not zero demand) | -2 steps | ADD | NO | +25% |

**Note: No actions taken for SKUs with stocks = 0**

## Price Tier Logic
- All tiers are **MARGINS** → Convert to price: `price = WAC / (1 - margin)`
- **Priority**: Market margins first → Internal margins extend range
- If at market_min and need lower → Use internal margin tiers below
- If at market_max and need higher → Use internal margin tiers above
- If at lowest/highest possible tier → Keep price unchanged
- If below commercial_min → Match minimum and add to SKU discount
- **Markup Fallback** (when no margin tiers): 15% of current margin per step
  - Example: SKU with 20% margin → each step = 3% price change

## Cart Rule (based on normal_refill)
- **OPEN (increase)**:
  - If current < normal_refill + stddev → jump to normal_refill + stddev
  - If current >= threshold → increase by 25%
  - Max cart rule: 150 (don't increase above)
- **RESTRICT (decrease)**:
  - Decrease by 25%
  - Minimum: normal_refill (or 2 if no refill data)

## Minimum Price Reduction
- Price reductions must be at least 0.25%


In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)


In [2]:
# =============================================================================
# CONFIGURATION
# =============================================================================

# Cart Rule Settings
MIN_CART_RULE = 2              # Minimum cart rule value
MIN_CART_CHANGE = 2            # Minimum change amount
CART_INCREASE_PCT = 0.25       # 25% increase (open)
CART_DECREASE_PCT = 0.25       # 25% decrease (restrict)

# Contribution Threshold (values are already in % format, e.g., 50 = 50%)
CONTRIBUTION_THRESHOLD = 50  # 50% threshold for discount removal

# Minimum Price Reduction
MIN_PRICE_REDUCTION_PCT = 0.0025  # 0.25% minimum reduction

# Input/Output Files
INPUT_FILE = 'pricing_with_discount.xlsx'
OUTPUT_FILE = f'pricing_actions_{datetime.now().strftime("%Y%m%d")}.xlsx'

print("✅ Configuration loaded")


✅ Configuration loaded


In [3]:
# =============================================================================
# LOAD DATA
# =============================================================================

df = pd.read_excel(INPUT_FILE)
print(f"✅ Loaded {len(df):,} rows from {INPUT_FILE}")
print(f"Columns: {len(df.columns)}")

# Quick validation
print(f"\nPerformance Status Distribution:")
print(df['combined_status'].value_counts())


✅ Loaded 28,382 rows from pricing_with_discount.xlsx
Columns: 130

Performance Status Distribution:
combined_status
No Data            7749
Critical           6029
Struggling         5960
Underperforming    2814
Over Achiever      2430
On Track           2265
Star Performer     1135
Name: count, dtype: int64


In [4]:
# =============================================================================
# HELPER: Build Price Tiers from Margins
# All tiers are MARGINS - convert to price: price = WAC / (1 - margin)
# Combines market margins + internal margins for full range
# =============================================================================

def build_price_tiers(row):
    """
    Build sorted list of price tiers from margins.
    Uses market margins first, then extends with internal margins.
    
    Returns: List of (price, tier_name) tuples sorted low to high
    """
    wac = row.get('wac_p', 0)
    
    if pd.isna(wac) or wac <= 0:
        return []
    
    tiers = []
    
    # Market margins (primary)
    market_cols = [
        ('below_market', 'below_market'),
        ('market_min', 'market_min'),
        ('market_25', 'market_25'),
        ('market_50', 'market_50'),
        ('market_75', 'market_75'),
        ('market_max', 'market_max'),
        ('above_market', 'above_market')
    ]
    
    # Internal margins (extend range if needed)
    internal_cols = [
        ('margin_tier_below', 'internal_below'),
        ('margin_tier_1', 'internal_1'),
        ('margin_tier_2', 'internal_2'),
        ('margin_tier_3', 'internal_3'),
        ('margin_tier_4', 'internal_4'),
        ('margin_tier_5', 'internal_5'),
        ('margin_tier_above_1', 'internal_above_1'),
        ('margin_tier_above_2', 'internal_above_2')
    ]
    
    # Collect all valid margins
    all_cols = market_cols + internal_cols
    
    for col, name in all_cols:
        margin = row.get(col)
        if pd.notna(margin) and 0 <= margin < 1:
            price = wac / (1 - margin)
            tiers.append((price, name, margin))
    
    # Sort by price (low to high) and remove duplicates
    tiers = sorted(set(tiers), key=lambda x: x[0])
    
    return tiers


def find_current_tier_index(current_price, tiers):
    """Find which tier index the current price is closest to."""
    if not tiers:
        return -1
    
    for i, (price, name, margin) in enumerate(tiers):
        if current_price <= price:
            return i
    
    return len(tiers) - 1


def get_price_at_steps(row, steps):
    """
    Get price after moving N steps in tiers.
    Negative steps = lower price, Positive steps = higher price
    
    Returns: (new_price, tier_name, hit_floor)
    """
    current_price = row.get('current_price', 0)
    commercial_min = row.get('commercial_min_price', 0) or 0
    wac = row.get('wac_p', 0)
    
    if pd.isna(current_price) or current_price <= 0:
        return current_price, 'invalid', False
    
    tiers = build_price_tiers(row)
    
    if not tiers:
        # No tiers - use markup fallback based on 15% of current margin
        # Current margin = (price - wac) / price
        if pd.notna(wac) and wac > 0 and current_price > wac:
            current_margin = (current_price - wac) / current_price
            markup_per_step = 0.1 * current_margin  # 15% of current margin
        else:
            markup_per_step = 0.03  # Fallback to 3% if no valid margin
        
        markup = steps * markup_per_step
        new_price = current_price * (1 + markup)
        return round(new_price, 2), f'markup_{markup*100:.1f}%', False
    
    # Find current position
    current_idx = find_current_tier_index(current_price, tiers)
    
    # Calculate new position
    new_idx = current_idx + steps
    new_idx = max(0, min(len(tiers) - 1, new_idx))
    
    new_price, tier_name, _ = tiers[new_idx]
    
    # Enforce minimum price reduction of 0.25% when reducing price
    if steps < 0 and new_price >= current_price:
        # Tier-based reduction wasn't enough, force minimum reduction
        min_reduced_price = current_price * (1 - MIN_PRICE_REDUCTION_PCT)
        if new_price > min_reduced_price:
            new_price = min_reduced_price
            tier_name = f'min_reduction_{MIN_PRICE_REDUCTION_PCT*100:.2f}%'
    elif steps < 0:
        # Check if tier-based reduction is less than minimum
        actual_reduction_pct = (current_price - new_price) / current_price
        if actual_reduction_pct < MIN_PRICE_REDUCTION_PCT:
            min_reduced_price = current_price * (1 - MIN_PRICE_REDUCTION_PCT)
            new_price = min_reduced_price
            tier_name = f'min_reduction_{MIN_PRICE_REDUCTION_PCT*100:.2f}%'
    
    # Check commercial minimum floor
    hit_floor = False
    if commercial_min > 0 and new_price < commercial_min:
        new_price = commercial_min
        tier_name = 'commercial_min'
        hit_floor = True
    
    return round(new_price, 2), tier_name, hit_floor


print("✅ Price tier functions defined")


✅ Price tier functions defined


In [5]:
# =============================================================================
# HELPER: Cart Rule Calculation
# =============================================================================

# Cart rule limits
MAX_CART_RULE = 150  # Don't increase above this

def adjust_cart_rule(current_cart, direction, normal_refill=0, refill_stddev=0):
    """
    Adjust cart rule based on normal_refill data.
    
    direction: 'open' (increase) or 'restrict' (decrease) or 'keep'
    
    OPEN (increase) rules:
    - If current < normal_refill + stddev → set to normal_refill + stddev
    - If current >= normal_refill + stddev → increase by 25%
    - If already above 150 → don't increase
    
    RESTRICT (decrease) rules:
    - Decrease by 25%
    - Minimum is normal_refill (or MIN_CART_RULE if no refill data)
    """
    current_cart = current_cart if pd.notna(current_cart) and current_cart > 0 else 999
    normal_refill = normal_refill if pd.notna(normal_refill) and normal_refill > 0 else 0
    refill_stddev = refill_stddev if pd.notna(refill_stddev) else 0
    
    # Target threshold = normal_refill + stddev
    target_threshold = normal_refill + refill_stddev if normal_refill > 0 else 0
    
    # Minimum cart rule (use normal_refill if available, else default MIN_CART_RULE)
    min_cart = max(MIN_CART_RULE, int(normal_refill)) if normal_refill > 0 else MIN_CART_RULE
    
    if direction == 'keep':
        return int(current_cart)
    
    if direction == 'open':
        # Don't increase if already above 150
        if current_cart >= MAX_CART_RULE:
            return int(current_cart)
        
        # If below threshold, jump to threshold
        if target_threshold > 0 and current_cart < target_threshold:
            new_cart = target_threshold
        else:
            # Already at or above threshold, increase by 25%
            change = max(MIN_CART_CHANGE, int(current_cart * CART_INCREASE_PCT))
            new_cart = current_cart + change
        
        # Cap at MAX_CART_RULE
        return min(MAX_CART_RULE, int(new_cart))
    
    elif direction == 'restrict':
        # Decrease by 25%
        change = max(MIN_CART_CHANGE, int(current_cart * CART_DECREASE_PCT))
        new_cart = current_cart - change
        
        # Enforce minimum (normal_refill or MIN_CART_RULE)
        return max(min_cart, int(new_cart))
    
    return int(current_cart)


print("✅ Cart rule function defined")


✅ Cart rule function defined


In [6]:
# =============================================================================
# HELPER: QD Tier Analysis
# =============================================================================

def get_qd_tier_to_remove(row):
    """
    Determine which QD tier to remove (highest first: T3 → T2 → T1).
    
    Returns: (tier_to_remove, qd_cntrb)
        tier_to_remove: 'T3', 'T2', 'T1', or None
        qd_cntrb: Total QD contribution from yesterday_qty_disc_cntrb
    """
    # Tier contributions from data_extraction output
    t1_cntrb = row.get('yesterday_t1_cntrb', 0) or 0
    t2_cntrb = row.get('yesterday_t2_cntrb', 0) or 0
    t3_cntrb = row.get('yesterday_t3_cntrb', 0) or 0
    
    # Use the total QD contribution directly from data
    qd_cntrb = row.get('yesterday_qty_disc_cntrb', 0) or 0
    
    # Check from highest tier down
    if t3_cntrb > 0:
        return 'T3', qd_cntrb
    elif t2_cntrb > 0:
        return 'T2', qd_cntrb
    elif t1_cntrb > 0:
        return 'T1', qd_cntrb
    else:
        return None, 0


print("✅ QD tier analysis function defined")


✅ QD tier analysis function defined


In [7]:
# =============================================================================
# MAIN: Action Engine Logic
# =============================================================================

def generate_action(row):
    """
    Generate pricing action for a single SKU-Warehouse.
    
    Conditions (in priority order):
    1. Zero Demand + Stock > 0
    2. Star/Over Achiever (not zero demand)
    3. On Track + Stock > 0 (not zero demand)
    4. Struggling/Underperforming/Critical + Stock > 0 (not zero demand)
    5. No Data + Stock > 0 (not zero demand)
    """
    # Get key values
    status = row.get('combined_status', '')
    yesterday_status = row.get('yesterday_status', '')
    stocks = row.get('stocks', 0) or 0
    zero_demand = row.get('zero_demand', 0) == 1
    oos_yesterday = row.get('oos_yesterday', 0) == 1
    current_price = row.get('current_price', 0)
    current_cart = row.get('current_cart_rule', 999)
    sku_disc_cntrb = row.get('yesterday_sku_disc_cntrb', 0) or 0
    
    # Get product sensitivity and refill data
    abc_class = row.get('abc_class', 'C')
    normal_refill = row.get('normal_refill', 0) or 0
    refill_stddev = row.get('refill_stddev', 0) or 0
    
    # Check for price oscillation prevention
    # If struggling today but was performing well yesterday → don't reduce price again
    was_performing_well_yesterday = yesterday_status in ['On Track', 'Over Achiever', 'Star Performer']
    # If performing well today but was struggling yesterday (and wasn't OOS) → don't increase price
    was_struggling_yesterday = yesterday_status in ['Struggling', 'Underperforming', 'Critical']
    
    # Get QD info
    tier_to_remove, qd_cntrb = get_qd_tier_to_remove(row)
    
    # Check if has existing discounts
    has_sku_disc = row.get('active_sku_disc_pct', 0) > 0
    has_qd = tier_to_remove is not None
    
    # Initialize result
    result = {
        'warehouse_id': row.get('warehouse_id'),
        'product_id': row.get('product_id'),
        'sku': row.get('sku'),
        'abc_class': abc_class,
        'current_price': current_price,
        'current_cart_rule': current_cart,
        'normal_refill': normal_refill,
        'refill_stddev': refill_stddev,
        'combined_status': status,
        'yesterday_status': yesterday_status,
        'oos_yesterday': oos_yesterday,
        'stocks': stocks,
        'zero_demand': zero_demand,
        'yesterday_sku_disc_cntrb': sku_disc_cntrb,
        'yesterday_qty_disc_cntrb': qd_cntrb,
        # Actions (to be filled)
        'new_price': current_price,
        'price_tier': None,
        'sku_discount_flag': 'KEEP',
        'qd_discount_flag': 'KEEP',
        'tier_with_problem': None,
        'new_cart_rule': current_cart,
        'action_reason': '',
        'hit_price_floor': False
    }
    
    # =========================================================================
    # CONDITION 1: Zero Demand + Stock > 0
    # =========================================================================
    if zero_demand and stocks > 0:
        # Price: -2 steps
        new_price, tier_name, hit_floor = get_price_at_steps(row, -2)
        result['new_price'] = new_price
        result['price_tier'] = tier_name
        result['hit_price_floor'] = hit_floor
        
        # SKU Discount: ADD
        result['sku_discount_flag'] = 'ADD'
        
        # QD: NO (don't add)
        result['qd_discount_flag'] = 'NO'
        result['tier_with_problem'] = None
        
        # Cart: open (based on normal_refill)
        result['new_cart_rule'] = adjust_cart_rule(current_cart, 'open', normal_refill, refill_stddev)
        
        result['action_reason'] = 'ZERO_DEMAND: Price -2 steps, Add SKU disc, No QD, Open cart'
        if hit_floor:
            result['action_reason'] += ' [HIT COMMERCIAL MIN - ADD TO SKU DISC]'
        
        return result
    
    # =========================================================================
    # CONDITION 2: Star Performer / Over Achiever (NOT zero demand) + Stock > 0
    # =========================================================================
    if status in ['Star Performer', 'Over Achiever'] and not zero_demand and stocks > 0:
        # Check contributions
        sku_high = sku_disc_cntrb > CONTRIBUTION_THRESHOLD
        qd_high = qd_cntrb > CONTRIBUTION_THRESHOLD
        
        # SKU Discount
        if sku_high:
            result['sku_discount_flag'] = 'REMOVE'
        else:
            result['sku_discount_flag'] = 'KEEP' if has_sku_disc else 'NO'
        
        # QD - Remove highest tier if contribution > 50%
        if qd_high and tier_to_remove:
            result['qd_discount_flag'] = f'REMOVE_{tier_to_remove}'
            result['tier_with_problem'] = tier_to_remove
        else:
            result['qd_discount_flag'] = 'KEEP' if has_qd else 'NO'
        
        # Price: +1 step only if BOTH contributions < 50%
        # BUT: Don't increase if yesterday was struggling and was NOT OOS (price was already reduced)
        skip_price_increase = was_struggling_yesterday and not oos_yesterday
        
        if not sku_high and not qd_high and not skip_price_increase:
            new_price, tier_name, _ = get_price_at_steps(row, +1)
            result['new_price'] = new_price
            result['price_tier'] = tier_name
        
        # Cart: restrict (based on normal_refill minimum)
        result['new_cart_rule'] = adjust_cart_rule(current_cart, 'restrict', normal_refill, refill_stddev)
        
        reasons = []
        reasons.append(f'{status.upper()}')
        if sku_high:
            reasons.append(f'SKU disc cntrb {sku_disc_cntrb:.0f}%>50% REMOVE')
        if qd_high and tier_to_remove:
            reasons.append(f'QD cntrb {qd_cntrb:.0f}%>50% REMOVE {tier_to_remove}')
        if skip_price_increase:
            reasons.append(f'NO PRICE INCREASE (yesterday={yesterday_status}, was in stock)')
        elif not sku_high and not qd_high:
            reasons.append('Both <50% Price +1 step')
        reasons.append('Restrict cart')
        result['action_reason'] = ' | '.join(reasons)
        
        return result
    
    # =========================================================================
    # CONDITION 3: On Track + Stock > 0 (NOT zero demand)
    # =========================================================================
    if status == 'On Track' and stocks > 0 and not zero_demand:
        # Price: +1 step
        # BUT: Don't increase if yesterday was struggling and was NOT OOS
        skip_price_increase = was_struggling_yesterday and not oos_yesterday
        
        if not skip_price_increase:
            new_price, tier_name, _ = get_price_at_steps(row, +1)
            result['new_price'] = new_price
            result['price_tier'] = tier_name
        
        # SKU Discount: KEEP if exists
        result['sku_discount_flag'] = 'KEEP' if has_sku_disc else 'NO'
        
        # QD: KEEP if exists
        result['qd_discount_flag'] = 'KEEP' if has_qd else 'NO'
        
        # Cart: KEEP
        result['new_cart_rule'] = adjust_cart_rule(current_cart, 'keep')
        
        if skip_price_increase:
            result['action_reason'] = f'ON_TRACK: NO PRICE INCREASE (yesterday={yesterday_status}, was in stock), Keep discounts, Keep cart'
        else:
            result['action_reason'] = 'ON_TRACK: Price +1 step, Keep discounts, Keep cart'
        
        return result
    
    # =========================================================================
    # CONDITION 4: Struggling / Underperforming / Critical + Stock > 0 (NOT zero demand)
    # Critical gets -2 steps, others get -1 step
    # BUT: Don't reduce price if yesterday was performing well (price was already increased)
    # =========================================================================
    if status in ['Struggling', 'Underperforming', 'Critical'] and stocks > 0 and not zero_demand:
        # Check if we should skip price reduction
        skip_price_reduction = was_performing_well_yesterday
        
        if skip_price_reduction:
            # Don't reduce price, just add to SKU discount
            result['sku_discount_flag'] = 'ADD'
            result['qd_discount_flag'] = 'KEEP' if has_qd else 'NO'
            result['new_cart_rule'] = adjust_cart_rule(current_cart, 'open', normal_refill, refill_stddev)
            result['action_reason'] = f'{status.upper()}: NO PRICE REDUCTION (yesterday={yesterday_status}), Add SKU disc only, Open cart'
        else:
            # Price: -2 steps for Critical, -1 step for others
            price_steps = -2 if status == 'Critical' else -1
            new_price, tier_name, hit_floor = get_price_at_steps(row, price_steps)
            result['new_price'] = new_price
            result['price_tier'] = tier_name
            result['hit_price_floor'] = hit_floor
            
            # SKU Discount: ADD
            result['sku_discount_flag'] = 'ADD'
            
            # QD: Keep existing or don't add
            result['qd_discount_flag'] = 'KEEP' if has_qd else 'NO'
            
            # Cart: open (based on normal_refill)
            result['new_cart_rule'] = adjust_cart_rule(current_cart, 'open', normal_refill, refill_stddev)
            
            result['action_reason'] = f'{status.upper()}: Price {price_steps} step(s), Add SKU disc, Open cart'
            if hit_floor:
                result['action_reason'] += ' [HIT COMMERCIAL MIN - ADD TO SKU DISC]'
        
        return result
    
    # =========================================================================
    # CONDITION 5: No Data + Stock > 0 (NOT zero demand)
    # =========================================================================
    if status == 'no_data' and stocks > 0 and not zero_demand:
        # Price: -2 steps
        new_price, tier_name, hit_floor = get_price_at_steps(row, -2)
        result['new_price'] = new_price
        result['price_tier'] = tier_name
        result['hit_price_floor'] = hit_floor
        
        # SKU Discount: ADD
        result['sku_discount_flag'] = 'ADD'
        
        # QD: Don't add
        result['qd_discount_flag'] = 'NO'
        
        # Cart: open (based on normal_refill)
        result['new_cart_rule'] = adjust_cart_rule(current_cart, 'open', normal_refill, refill_stddev)
        
        result['action_reason'] = 'NO_DATA: Price -2 steps, Add SKU disc, Open cart'
        if hit_floor:
            result['action_reason'] += ' [HIT COMMERCIAL MIN - ADD TO SKU DISC]'
        
        return result
    
    # =========================================================================
    # DEFAULT: No action (no stock or other unhandled cases)
    # =========================================================================
    result['action_reason'] = f'NO_ACTION: Status={status}, Stocks={stocks}, ZeroDemand={zero_demand}'
    result['sku_discount_flag'] = 'NO'
    result['qd_discount_flag'] = 'NO'
    
    return result


print("✅ Action engine function defined")


✅ Action engine function defined


In [8]:
# =============================================================================
# EXECUTE: Generate Actions for All SKUs
# =============================================================================

print(f"Processing {len(df):,} SKU-Warehouse combinations...")

actions = []
total = len(df)

for idx, row in df.iterrows():
    if (idx + 1) % 10000 == 0:
        print(f"  Progress: {idx + 1:,}/{total:,} ({(idx+1)/total*100:.1f}%)")
    
    action = generate_action(row)
    actions.append(action)

results_df = pd.DataFrame(actions)
print(f"\n✅ Generated {len(results_df):,} actions")


Processing 28,382 SKU-Warehouse combinations...
  Progress: 10,000/28,382 (35.2%)
  Progress: 20,000/28,382 (70.5%)

✅ Generated 28,382 actions


In [9]:
# =============================================================================
# SUMMARY
# =============================================================================

print("=" * 70)
print("ACTIONS SUMMARY")
print("=" * 70)

# By condition (from action_reason)
print("\n📊 BY CONDITION:")
conditions = results_df['action_reason'].str.split(':').str[0].value_counts()
for cond, cnt in conditions.items():
    print(f"   {cond}: {cnt:,} ({cnt/len(results_df)*100:.1f}%)")

# SKU Discount Actions
print("\n💰 SKU DISCOUNT FLAGS:")
for flag, cnt in results_df['sku_discount_flag'].value_counts().items():
    print(f"   {flag}: {cnt:,} ({cnt/len(results_df)*100:.1f}%)")

# QD Actions
print("\n📦 QD DISCOUNT FLAGS:")
for flag, cnt in results_df['qd_discount_flag'].value_counts().items():
    print(f"   {flag}: {cnt:,} ({cnt/len(results_df)*100:.1f}%)")

# QD Tier Problems
print("\n⚠️ QD TIERS TO REMOVE:")
tier_problems = results_df['tier_with_problem'].value_counts(dropna=False)
for tier, cnt in tier_problems.items():
    label = tier if pd.notna(tier) else 'None'
    print(f"   {label}: {cnt:,}")

# Price Changes
price_changes = (results_df['new_price'] != results_df['current_price']).sum()
price_up = (results_df['new_price'] > results_df['current_price']).sum()
price_down = (results_df['new_price'] < results_df['current_price']).sum()
print(f"\n📈 PRICE CHANGES: {price_changes:,} total")
print(f"   Increase: {price_up:,}")
print(f"   Decrease: {price_down:,}")

# Hit Floor
hit_floor = results_df['hit_price_floor'].sum()
print(f"\n🔻 HIT COMMERCIAL MINIMUM: {hit_floor:,}")


ACTIONS SUMMARY

📊 BY CONDITION:
   NO_ACTION: 9,802 (34.5%)
   STRUGGLING: 4,489 (15.8%)
   ZERO_DEMAND: 4,159 (14.7%)
   CRITICAL: 3,564 (12.6%)
   UNDERPERFORMING: 2,220 (7.8%)
   ON_TRACK: 1,748 (6.2%)
   OVER ACHIEVER | Both <50% Price +1 step | Restrict cart: 1,133 (4.0%)
   STAR PERFORMER | Both <50% Price +1 step | Restrict cart: 567 (2.0%)
   OVER ACHIEVER | SKU disc cntrb 100%>50% REMOVE | Restrict cart: 119 (0.4%)
   OVER ACHIEVER | NO PRICE INCREASE (yesterday=Struggling, was in stock) | Restrict cart: 95 (0.3%)
   OVER ACHIEVER | NO PRICE INCREASE (yesterday=Critical, was in stock) | Restrict cart: 76 (0.3%)
   STAR PERFORMER | SKU disc cntrb 100%>50% REMOVE | Restrict cart: 47 (0.2%)
   OVER ACHIEVER | NO PRICE INCREASE (yesterday=Underperforming, was in stock) | Restrict cart: 41 (0.1%)
   OVER ACHIEVER | SKU disc cntrb 100%>50% REMOVE | NO PRICE INCREASE (yesterday=Struggling, was in stock) | Restrict cart: 29 (0.1%)
   OVER ACHIEVER | SKU disc cntrb 100%>50% REMOVE | N

In [10]:
# =============================================================================
# EXPORT TO EXCEL
# =============================================================================

# Reorder columns for output
output_cols = [
    # Identifiers
    'warehouse_id', 'product_id', 'sku',
    # Product info
    'abc_class',
    # Current state
    'current_price', 'current_cart_rule', 'normal_refill', 'refill_stddev',
    'combined_status', 'yesterday_status', 'oos_yesterday',
    'stocks', 'zero_demand',
    # Contributions (from data_extraction)
    'yesterday_sku_disc_cntrb', 'yesterday_qty_disc_cntrb',
    # Actions
    'new_price', 'price_tier', 'hit_price_floor',
    'sku_discount_flag', 
    'qd_discount_flag', 'tier_with_problem',
    'new_cart_rule',
    'action_reason'
]

output_df = results_df[output_cols]

# Save to Excel
output_df.to_excel(OUTPUT_FILE, index=False)
print(f"✅ Saved to {OUTPUT_FILE}")
print(f"   Total rows: {len(output_df):,}")


✅ Saved to pricing_actions_20260125.xlsx
   Total rows: 28,382


In [11]:
# =============================================================================
# SAMPLE OUTPUT
# =============================================================================

print("Sample Actions by Condition:\n")

# Sample Zero Demand
zero_dem = output_df[output_df['action_reason'].str.contains('ZERO_DEMAND', na=False)].head(3)
if len(zero_dem) > 0:
    print("📍 Zero Demand:")
    display(zero_dem)

# Sample Star/Over Achiever
star = output_df[output_df['action_reason'].str.contains('STAR|OVER', na=False)].head(3)
if len(star) > 0:
    print("\n📍 Star/Over Achiever:")
    display(star)

# Sample On Track
on_track = output_df[output_df['action_reason'].str.contains('ON_TRACK', na=False)].head(3)
if len(on_track) > 0:
    print("\n📍 On Track:")
    display(on_track)

# Sample Struggling/Underperforming/Critical
struggling = output_df[output_df['action_reason'].str.contains('STRUGGLING|UNDERPERFORMING|CRITICAL', na=False)].head(3)
if len(struggling) > 0:
    print("\n📍 Struggling/Underperforming/Critical:")
    display(struggling)


Sample Actions by Condition:

📍 Zero Demand:


,warehouse_id,product_id,sku,abc_class,current_price,current_cart_rule,normal_refill,refill_stddev,combined_status,yesterday_status,oos_yesterday,stocks,zero_demand,yesterday_sku_disc_cntrb,yesterday_qty_disc_cntrb,new_price,price_tier,hit_price_floor,sku_discount_flag,qd_discount_flag,tier_with_problem,new_cart_rule,action_reason
37,337,20196,تويتش مشروب شعيرغازي بطعم الاناناس - 330 مل,C,99.75,10,1.85,1.46,Critical,No Data,False,29,True,0.0,0.0,99.50,min_reduction_0.25%,False,ADD,NO,None,12,"ZERO_DEMAND: Price -2 steps, Add SKU disc, No ..."
38,8,20196,تويتش مشروب شعيرغازي بطعم الاناناس - 330 مل,C,99.75,10,1.67,0.78,Critical,No Data,False,15,True,0.0,0.0,99.50,min_reduction_0.25%,False,ADD,NO,None,12,"ZERO_DEMAND: Price -2 steps, Add SKU disc, No ..."
67,337,1234,سمن جنة ظرف - 350 جم,C,457.00,5,1.00,0.00,Critical,No Data,False,8,True,0.0,0.0,452.74,internal_below,False,ADD,NO,None,7,"ZERO_DEMAND: Price -2 steps, Add SKU disc, No ..."



📍 Star/Over Achiever:


,warehouse_id,product_id,sku,abc_class,current_price,current_cart_rule,normal_refill,refill_stddev,combined_status,yesterday_status,oos_yesterday,stocks,zero_demand,yesterday_sku_disc_cntrb,yesterday_qty_disc_cntrb,new_price,price_tier,hit_price_floor,sku_discount_flag,qd_discount_flag,tier_with_problem,new_cart_rule,action_reason
5,236,12003,الضحى مكرونة بصوص ماك آند تشيز - 175 جم,C,25.50,10,5.62,8.43,Star Performer,Over Achiever,False,59,False,35.29,0.0,25.86,internal_5,False,KEEP,NO,None,8,STAR PERFORMER | Both <50% Price +1 step | Res...
7,236,21459,سولا فواكة - 28 قطعه,C,11.75,10,7.04,9.83,Over Achiever,Struggling,False,87,False,100.00,0.0,11.75,None,False,REMOVE,NO,None,8,OVER ACHIEVER | SKU disc cntrb 100%>50% REMOVE...
9,501,13490,مكرونة هلالية بساطة - 1 كجم,C,241.00,13,1.00,0.00,Over Achiever,No Data,False,62,False,0.00,0.0,243.82,internal_5,False,KEEP,NO,None,10,OVER ACHIEVER | Both <50% Price +1 step | Rest...



📍 On Track:


,warehouse_id,product_id,sku,abc_class,current_price,current_cart_rule,normal_refill,refill_stddev,combined_status,yesterday_status,oos_yesterday,stocks,zero_demand,yesterday_sku_disc_cntrb,yesterday_qty_disc_cntrb,new_price,price_tier,hit_price_floor,sku_discount_flag,qd_discount_flag,tier_with_problem,new_cart_rule,action_reason
1,401,972,نواعم بسكويت- 5 ج,B,49.00,695,4.41,3.89,On Track,On Track,False,35,False,0.0,0.0,49.39,internal_2,False,KEEP,NO,None,695,"ON_TRACK: Price +1 step, Keep discounts, Keep ..."
19,236,8191,عسل نحل البوادى زهرة الموالح - 235 جم,C,40.25,24,3.50,2.51,On Track,No Data,False,54,False,0.0,0.0,40.61,internal_4,False,KEEP,NO,None,24,"ON_TRACK: Price +1 step, Keep discounts, Keep ..."
20,962,8191,عسل نحل البوادى زهرة الموالح - 235 جم,C,40.25,24,3.13,1.59,On Track,Over Achiever,False,33,False,100.0,0.0,40.61,internal_4,False,KEEP,NO,None,24,"ON_TRACK: Price +1 step, Keep discounts, Keep ..."



📍 Struggling/Underperforming/Critical:


,warehouse_id,product_id,sku,abc_class,current_price,current_cart_rule,normal_refill,refill_stddev,combined_status,yesterday_status,oos_yesterday,stocks,zero_demand,yesterday_sku_disc_cntrb,yesterday_qty_disc_cntrb,new_price,price_tier,hit_price_floor,sku_discount_flag,qd_discount_flag,tier_with_problem,new_cart_rule,action_reason
0,401,10413,مربى البوادى مشمش - 365 جم,C,27.75,12,6.80,4.66,Critical,No Data,False,85,False,0.0,0.0,27.23,internal_below,False,ADD,NO,None,15,"CRITICAL: Price -2 step(s), Add SKU disc, Open..."
2,337,11726,مناديل زينة تريو تواليت عرض 5 + 1 رول هدية 6 بكرة,C,302.50,25,2.06,1.46,Struggling,No Data,False,27,False,0.0,0.0,301.74,min_reduction_0.25%,False,ADD,NO,None,31,"STRUGGLING: Price -1 step(s), Add SKU disc, Op..."
3,8,11726,مناديل زينة تريو تواليت عرض 5 + 1 رول هدية 6 بكرة,C,302.50,25,1.75,1.17,Underperforming,No Data,False,26,False,0.0,0.0,301.74,min_reduction_0.25%,False,ADD,NO,None,31,"UNDERPERFORMING: Price -1 step(s), Add SKU dis..."
